In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sys

In [11]:
sys.path.append('/Users/hughcross/Analysis/repos/seq_tools/seq_functions')
from seq_funcs import extract_seqs

### Import sql table

In [2]:
sqlite_file = '/Users/hughcross/Analysis/CAMEL/camel2016/Camel_Plant_Reference_db.sqlite'

In [3]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [4]:
df = pd.read_sql('select * from plant_APNI_taxonomy', conn)
df.head()

,Taxon,TaxonCode,Tax_ID,regnum,division,class,subclass,superorder,order,family,subfamily,tribe,genus,species,subspecies,varietas
0,Anthocercis intricata,Anthocercis_intricata_195720,195720,Plantae,Charophyta,Equisetopsida,Magnoliidae,Asteranae,Solanales,Solanaceae,Nicotianoideae,Anthocercideae,Anthocercis,intricata,None,None
1,Myoporum platycarpum,Myoporum_platycarpum_796003,796003,Plantae,Charophyta,Equisetopsida,Magnoliidae,Asteranae,Lamiales,Scrophulariaceae,None,Myoporeae,Myoporum,platycarpum,None,None
2,Saphesia flaccida,Saphesia_flaccida_216014,216014,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Aizoaceae,None,None,Saphesia,flaccida,None,None
3,Spyridium ulicinum,Spyridium_ulicinum_323969,323969,Plantae,Charophyta,Equisetopsida,Magnoliidae,Rosanae,Rosales,Rhamnaceae,None,Pomaderreae,Spyridium,ulicinum,None,None
4,Austrostipa wakoolica,Austrostipa_wakoolica_1030042,1030042,Plantae,Charophyta,Equisetopsida,Magnoliidae,Lilianae,Poales,Poaceae,Pooideae,Stipeae,Austrostipa,wakoolica,None,None


## Get chenopod genera

In [5]:
# make new df with just chenopods
## output all rows of a specific column value
df_cheno = df.loc[df.family == 'Chenopodiaceae']

In [6]:
df_cheno.head()

,Taxon,TaxonCode,Tax_ID,regnum,division,class,subclass,superorder,order,family,subfamily,tribe,genus,species,subspecies,varietas
51,Suaeda heteroptera,Suaeda_heteroptera_224170,224170,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Chenopodiaceae,Suaedoideae,None,Suaeda,heteroptera,None,None
68,Suaeda aralocaspica,Suaeda_aralocaspica_224144,224144,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Chenopodiaceae,Suaedoideae,None,Suaeda,aralocaspica,None,None
77,Atriplex rosea,Atriplex_rosea_3552,3552,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Chenopodiaceae,Chenopodioideae,Atripliceae,Atriplex,rosea,None,None
102,Suaeda linifolia,Suaeda_linifolia_224172,224172,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Chenopodiaceae,Suaedoideae,None,Suaeda,linifolia,None,None
125,Maireana campanulata,Maireana_campanulata_381269,381269,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Chenopodiaceae,Camphorosmoideae,Camphorosmeae,Maireana,campanulata,None,None


In [7]:
# list of all Fabaceae genera from df
chen_genera = df_cheno['genus'].tolist()
cheno_genset = set(chen_genera)
print(len(chen_genera))
print(len(cheno_genset))

509
50


## Parse uc cluster file and pull out only chenopod hits

In [8]:
uc_file = open('/Users/hughcross/Analysis/CAMEL/cam2018/its_q2_trial1/clustout1.uc')

In [9]:
hits = []
for line in uc_file:
    if line.startswith('H'):
        parts = line.split('\t')
        query = parts[8]
        ref = parts[9]
        genus = ref.split('_')[0]
        if genus in cheno_genset:
            hits.append(query)
uc_file.close()
print(len(hits))

7085


In [12]:
hitset = set(hits)

In [13]:
# function to make new sequence file
extract_seqs(hitset, '/Users/hughcross/Analysis/CAMEL/cam2018/its_q2_trial1/cam_all_its_qii.fasta', '/Users/hughcross/Analysis/CAMEL/cam2018/its_q2_trial1/cheno_its.fasta')

## Repeat with trnL sequences

```
vsearch --usearch_global cam_all_trnl_qii.fasta \
  -db /Users/hughcross/Analysis/CAMEL/cam2018/refs_trnl/trimd_trnl_all_ref_seqs_5jan18.fasta \
  -id 0.95 --uc clustout_trnL.uc
 ```

In [14]:
uc_trnl = open('/Users/hughcross/Analysis/CAMEL/cam2018/char_based_trial/clustout_trnL.uc')

In [15]:
hits_tl = []
for line in uc_trnl:
    if line.startswith('H'):
        parts = line.split('\t')
        query = parts[8]
        ref = parts[9]
        genus = ref.split('_')[0]
        if genus in cheno_genset:
            hits_tl.append(query)
uc_trnl.close()
print(len(hits_tl))

3765


In [16]:
tlset = set(hits_tl)
extract_seqs(hits_tl, '/Users/hughcross/Analysis/CAMEL/cam2018/char_based_trial/cam_all_trnl_qii.fasta', '/Users/hughcross/Analysis/CAMEL/cam2018/char_based_trial/cam_trnl_cheno.fasta')

### Map to reference with BWA

`bwa mem -E [2,2] Atriplex_vesicaria_AD_b74_ref_seq.fasta cam_trnl_cheno.fasta > chenos_mapped.sam`

In [39]:
clade_str_regex

{'Bassia': 'CTTATTTCTG[AG]AG[AG]A[AT]A----',
 'Salsola': 'CTTCTACCTGGAGAAGA----',
 'Suaeda': 'TTGATTTTTGCAGGATA----',
 'Tecticornia': 'CTTATACCTTAAGGAA-----',
 'bluebush': 'CTTATTCCTGGA[GT]AATA----',
 'saltbush': 'C[CT]T[AC][ACGT][CT][CT]C[GT]GC[AG][GT]A[AC][ACGT][ACGT][ACGT][ACGT][ACGT][ACGT]'}